### **Install dependencies**

In [1]:
# % pip install -r requirements.txt

### **Importing the functions from python files**

In [2]:
from ExtractMatchData._functions.get_matches_url import get_matches_url
from ExtractMatchData._functions.get_match_id import get_match_ids
from ExtractMatchData._functions.get_request import get_request
from ExtractMatchData._functions.download_files import download_files
from ExtractMatchData._functions.unrar import unrar
from ExtractMatchData._functions.extrair_dados import run_csda_on_demos
from ExtractMatchData._ETL.merge_files import merge_csv_files
import os
from ExtractMatchData._functions.chromelib import pd

# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)  # Disable line wrapping
# pd.set_option('display.max_colwidth', None)  # Show full column content

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

# -------------------------------------------------------------------------
#   The ID of each event can be found in HLTV in either of the links below
#   For example: 
#   https://www.hltv.org/events/7909/blast-bounty-2025-season-1-finals
#   https://www.hltv.org/results?event=7909
# -------------------------------------------------------------------------

# id_event = [ 7909 , 8229]
id_event = [
    #8043,
    # 8034,
    # 8229,
    7909,
    # 7524,
    # 7861,
    # 7557,
    7603
    ]


### **ACTUAL SCRAPING**

In [3]:
root = os.path.join(os.getcwd(), 'ExtractMatchData') # Get the root directory

# Gets the URLs from all the matches in the event and other info into a pandas dataframe
tournaments_df = get_matches_url(*id_event, root=root)

# Get an ID for each match used in a url
tournaments_df = get_match_ids(tournaments_df)

# Makes a get request to fetch the direct links to download the demos
tournaments_df = get_request(tournaments_df)
                           
# Makes the request to fetch the direct link to download the demos
tournaments_df = download_files(tournaments_df)

unrar(tournaments_df)

run_csda_on_demos(tournaments_df)

match_data = merge_csv_files(tournaments_df)

---------------------------------------------
       MATCHUPS CAN BE Bo1, Bo3 or Bo5
---------------------------------------------

Event: BLAST Bounty 2025 Season 1 Finals
Number of matchups: 7

Event: ESL Challenger Katowice 2024
Number of matchups: 13


Total number of matchups: 20
---------------------------------------------


Exctracting .rar   : 100%|███████████████████████████████████████████████████████████| 20/20 [01:52]



---------------------------------------------

Number of games: 18


Processing BLAST Bounty 2025 Season 1 Finals: 100%|██████████████████████████████████| 18/18 [03:18]



Number of games: 23


Processing ESL Challenger Katowice 2024: 100%|███████████████████████████████████████| 23/23 [04:26]


In [4]:
# Access individual DataFrames
matches_df = match_data['_match']
teams_df = match_data['_teams']
kills_df = match_data['_kills']
players_df = match_data['_players']
players_economy_df = match_data['_players_economy']
clutches_df = match_data['_clutches']

In [5]:
print("Tournaments:")
display(tournaments_df.head(5))

print("\nMatches:")
display(matches_df.head(5))

print("\nTeams:")
display(teams_df.head(5))

print("\nKills:")
display(kills_df.head(5))

print("\nPlayers:")
display(players_df.head(5))

print("\nPlayers Economy:")
display(players_economy_df.head(5))

print("\nClutches:")
display(clutches_df.head(5))

Tournaments:


,root,url,url_event,tournament,nspc_tournament,match_id,url_request,url_demo,file_name,demo_path,output_data_path
0,c:\cs2-hltv-event-scrapper\ExtractMatchData,https://www.hltv.org/matches/2378460/eternal-f...,https://www.hltv.org/results?event=7909,BLAST Bounty 2025 Season 1 Finals,blast-bounty-2025-season-1-finals,93701,https://www.hltv.org/download/demo/93701,https://r2-demos.hltv.org/demos/111377/blast-b...,blast-bounty-2025-season-1-finals-eternal-fire...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...
1,c:\cs2-hltv-event-scrapper\ExtractMatchData,https://www.hltv.org/matches/2378459/natus-vin...,https://www.hltv.org/results?event=7909,BLAST Bounty 2025 Season 1 Finals,blast-bounty-2025-season-1-finals,93689,https://www.hltv.org/download/demo/93689,https://r2-demos.hltv.org/demos/111365/blast-b...,blast-bounty-2025-season-1-finals-natus-vincer...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...
2,c:\cs2-hltv-event-scrapper\ExtractMatchData,https://www.hltv.org/matches/2378458/g2-vs-ete...,https://www.hltv.org/results?event=7909,BLAST Bounty 2025 Season 1 Finals,blast-bounty-2025-season-1-finals,93686,https://www.hltv.org/download/demo/93686,https://r2-demos.hltv.org/demos/111362/blast-b...,blast-bounty-2025-season-1-finals-g2-vs-eterna...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...
3,c:\cs2-hltv-event-scrapper\ExtractMatchData,https://www.hltv.org/matches/2378455/natus-vin...,https://www.hltv.org/results?event=7909,BLAST Bounty 2025 Season 1 Finals,blast-bounty-2025-season-1-finals,93650,https://www.hltv.org/download/demo/93650,https://r2-demos.hltv.org/demos/111327/blast-b...,blast-bounty-2025-season-1-finals-natus-vincer...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...
4,c:\cs2-hltv-event-scrapper\ExtractMatchData,https://www.hltv.org/matches/2378456/vitality-...,https://www.hltv.org/results?event=7909,BLAST Bounty 2025 Season 1 Finals,blast-bounty-2025-season-1-finals,93643,https://www.hltv.org/download/demo/93643,https://r2-demos.hltv.org/demos/111318/blast-b...,blast-bounty-2025-season-1-finals-vitality-vs-...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...,c:\cs2-hltv-event-scrapper\ExtractMatchData\do...



Matches:


,checksum,date,map,tournament
0,59e5b2c76e05bce0,2025-01-26,de_anubis,BLAST Bounty 2025 Season 1 Finals
1,2a4c3f36d8d46afa,2025-01-26,de_nuke,BLAST Bounty 2025 Season 1 Finals
2,d8b7e0bddd1b3eb3,2025-01-26,de_dust2,BLAST Bounty 2025 Season 1 Finals
3,9cb48de6a17ebe1,2025-01-26,de_mirage,BLAST Bounty 2025 Season 1 Finals
4,f101e58241ef725a,2025-01-23,de_anubis,BLAST Bounty 2025 Season 1 Finals



Teams:


...

,name,steamid,team_name,kills,assists,deaths,headshots,hs_%,kd,kast,...,first_death,trade_kill,trade_death,1k,2k,3k,4k,5k,match_checksum,tournament
0,XANTARES,76561198044118796,Eternal Fire,23,10,21,14,60,1.095238,76.666664,...,4,3,7,8,4,1,1,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
1,zont1x,76561198995880877,Team Spirit,24,9,22,7,29,1.090909,76.666664,...,5,3,8,9,4,1,1,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
2,jottAAA,76561198410750263,Eternal Fire,20,7,21,13,65,0.952381,66.666672,...,1,5,1,12,4,0,0,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
3,woxic,76561198083485506,Eternal Fire,25,6,11,10,40,2.272727,86.666664,...,1,3,1,12,5,1,0,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
4,Wicadia,76561198812513923,Eternal Fire,20,4,22,10,50,0.909091,56.666668,...,5,3,3,6,4,2,0,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals



Players Economy:


,steamid,name,player_side,equipment_value,type,match_checksum,tournament
0,76561198045898864,chopper,Counter Terrorist,400,pistol,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
1,76561198081484775,sh1ro,Counter Terrorist,950,pistol,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
2,76561199063238565,magixx,Counter Terrorist,900,pistol,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
3,76561198995880877,zont1x,Counter Terrorist,1150,pistol,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
4,76561197967432889,MAJ3R,Terrorist,850,pistol,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals



Clutches:


,round,won,steamid,name,survived,kill_count,match_checksum,tournament
0,1,0,76561199063238565,magixx,0,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
1,2,0,76561198081484775,sh1ro,0,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
2,3,0,76561198083485506,woxic,1,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
3,4,0,76561198083485506,woxic,1,0,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals
4,5,0,76561198812513923,Wicadia,0,1,59e5b2c76e05bce0,BLAST Bounty 2025 Season 1 Finals


In [8]:
output_dir = os.path.join(os.getcwd(), "ExtractMatchData" ,"tournaments_tables")
os.makedirs(output_dir, exist_ok=True)

# Export tables to csv
tournaments_df.to_csv(os.path.join(output_dir, "tournaments.csv"), index=False)
print(f"Saved {os.path.join(output_dir, 'tournaments.csv')}")

for name, df in match_data.items():
    output_file = os.path.join(output_dir, f"{name.strip('_')}.csv")
    df.to_csv(output_file, index=False)
    print(f"Saved {output_file}")
 
# Just exporting the tables in markdown format  
output_file = os.path.join(output_dir, "all_tables.txt")
with open(output_file, 'w') as f:
    # First write tournaments table
    f.write("\nTournaments Table:\n")
    f.write('| ' + ' | '.join(tournaments_df.columns) + ' |\n')
    f.write('|' + '|'.join(['---' for _ in tournaments_df.columns]) + '|\n')
    first_row = tournaments_df.iloc[0].astype(str)
    f.write('| ' + ' | '.join(first_row) + ' |\n\n')
    
    # Then write all other tables
    for name, df in match_data.items():
        # Write table name as header
        f.write(f"\n{name.strip('_')} Table:\n")
        # Write header row
        f.write('| ' + ' | '.join(df.columns) + ' |\n')
        # Write separator row
        f.write('|' + '|'.join(['---' for _ in df.columns]) + '|\n')
        # Write first data row only
        first_row = df.iloc[0].astype(str)
        f.write('| ' + ' | '.join(first_row) + ' |\n')
        f.write('\n') # Add blank line between tables
    print(f"Saved all tables to {output_file}")

Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\tournaments.csv
Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\clutches.csv
Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\kills.csv
Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\match.csv
Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\players.csv
Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\players_economy.csv
Saved c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\teams.csv
Saved all tables to c:\cs2-hltv-event-scrapper\ExtractMatchData\tournaments_tables\all_tables.txt
